In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import base64
import pandas as pd
JupyterDash.infer_jupyter_proxy_config()

# Import your CRUD Python module
from CRUD_Python_Module import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "SNHU1234"

# Connect to MongoDB
db = AnimalShelter()

# Load all data from the MongoDB collection
df = pd.DataFrame.from_records(db.read({}))
df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Grazioso Salvare logo
image_filename = 'Grazioso Salvare Logo.png'  # Replace with your file path
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# App Layout
app.layout = html.Div([
    html.Center(html.A(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                         style={'height': '100px'}),
                       href='http://www.snhu.edu')),  # Added clickable URL
    html.Center(html.B(html.H1('Grazioso Salvare Dashboard - Daniel Urena'))),
    html.Hr(),

    # Filtering options
    html.Div([
        html.Label('Select Rescue Type:'),
        dcc.Dropdown(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            placeholder="Select a rescue type",
            style={'width': '50%'}
        )
    ]),
    html.Br(),

    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left'},
        sort_action='native',
        filter_action='native',
        row_selectable='single',
        selected_rows=[],
        style_data={'whiteSpace': 'normal', 'height': 'auto'},
    ),

    html.Br(),
    html.Hr(),

    # Charts
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Filter function based on rescue type
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    # -------------------------------
    # UPDATED: Correct min/max ages
    # -------------------------------
    if filter_type == 'Water Rescue':
        query = {"$and": [
            {"breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}},
            {"sex_upon_outcome": "Intact Female"},
            {"age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}  # min/max age corrected
        ]}
    elif filter_type == 'Mountain or Wilderness Rescue':
        query = {"$and": [
            {"breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog",
                               "Siberian Husky", "Rottweiler"]}},
            {"sex_upon_outcome": "Intact Male"},
            {"age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}  # min/max age corrected
        ]}
    elif filter_type == 'Disaster or Individual Tracking':
        query = {"$and": [
            {"breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever",
                               "Bloodhound", "Rottweiler"]}},
            {"sex_upon_outcome": "Intact Male"},
            {"age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}}  # corrected upper age
        ]}
    else:  # Reset
        query = {}  # Return all records

    dff = pd.DataFrame.from_records(db.read(query))
    if '_id' in dff.columns:
        dff.drop(columns=['_id'], inplace=True)
    return dff.to_dict('records')


# Chart callback
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData is None:
        dff = df
    else:
        dff = pd.DataFrame.from_dict(viewData)

    fig = px.pie(dff, names='breed', title='Breed Distribution')
    return [dcc.Graph(figure=fig)]


# Map callback
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None:
        return
    dff = pd.DataFrame.from_dict(viewData)
    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
                dl.Tooltip(dff.iloc[row, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, 9])
                ])
            ])
        ])
    ]


# Highlight selected columns
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{'if': {'column_id': i}, 'background_color': '#D2F3FF'} for i in selected_columns]


# Run the app
app.run_server(mode='external', port=8053)

Dash app running on https://normalathlete-jeepappear-3000.codio.io/proxy/8053/
